<a href="https://colab.research.google.com/github/medhhaa/Bank-Churn-Prediction/blob/main/Bank_Churner_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
A business manager of a consumer credit card portfolio is facing the problem of customer attrition. They want to analyze the data to find out the reason behind this and leverage the same to predict customers who are likely to drop off.

This dataset is sourced from https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers/data

A manager at the bank is disturbed with more and more customers leaving their credit card services. They would really appreciate if one could predict for them who is gonna get churned so they can proactively go to the customer to provide them better services and turn customers' decisions in the opposite direction

Now, this dataset consists of 10,000 customers mentioning their age, salary, marital_status, credit card limit, credit card category, etc. There are nearly 18 features.

We have only 16.07% of customers who have churned. Thus, it's a bit difficult to train our model to predict churning customers.


### 1.1 Import Libraries

In [2]:
# Import Libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Interactive visualizations
import plotly.express as ex
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import plotly.io as pio
pio.renderers.default = 'colab'

# sns.set_style('darkgrid')
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score as f1
from sklearn.metrics import confusion_matrix
# import scikitplot as skplt

plt.rc('figure',figsize=(18,9))




In [3]:
# %pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

### 1.2 Load Data

In [4]:
df = pd.read_csv('BankChurners.csv')
df = df[df.columns[:-2]]
df.head(5)

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 10127 non-null  int64  
 1   Attrition_Flag            10127 non-null  object 
 2   Customer_Age              10127 non-null  int64  
 3   Gender                    10127 non-null  object 
 4   Dependent_count           10127 non-null  int64  
 5   Education_Level           10127 non-null  object 
 6   Marital_Status            10127 non-null  object 
 7   Income_Category           10127 non-null  object 
 8   Card_Category             10127 non-null  object 
 9   Months_on_book            10127 non-null  int64  
 10  Total_Relationship_Count  10127 non-null  int64  
 11  Months_Inactive_12_mon    10127 non-null  int64  
 12  Contacts_Count_12_mon     10127 non-null  int64  
 13  Credit_Limit              10127 non-null  float64
 14  Total_

Inference: The data does not have any missing values.

### Understanding Dataset

- **CLIENTNUM**: Client number. Unique identifier for the customer holding the account  
- **Attrition_Flag**: Internal event (customer activity) variable – if the account is closed then 1 else 0  
- **Customer_Age**: Demographic variable – customer’s age in years  
- **Gender**: Demographic variable – M = Male, F = Female  
- **Dependent_count**: Demographic variable – number of dependents  
- **Education_Level**: Demographic variable – highest educational qualification of the account holder (e.g., high school, graduate, etc.)  
- **Marital_Status**: Demographic variable – Married, Single, Divorced, Unknown  
- **Income_Category**: Demographic variable – annual income bracket (< $40K, $40K–$60K, $60K–$80K, $80K–$120K, > $120K)  
- **Card_Category**: Product variable – type of card (Blue, Silver, Gold, Platinum)  
- **Months_on_book**: Relationship variable – number of months the customer has held the account  
- **Total_Relationship_Count**: Relationship variable – total number of products held by the customer  
- **Months_Inactive_12_mon**: Activity variable – number of months inactive in the last 12 months  
- **Contacts_Count_12_mon**: Engagement variable – number of contacts in the last 12 months  
- **Credit_Limit**: Financial variable – credit limit on the credit card  
- **Total_Revolving_Bal**: Financial variable – total revolving balance on the credit card  
- **Avg_Open_To_Buy**: Financial variable – average open-to-buy credit line over the last 12 months  
- **Total_Amt_Chng_Q4_Q1**: Behavioral variable – change in transaction amount (Q4 over Q1)  
- **Total_Trans_Amt**: Behavioral variable – total transaction amount in the last 12 months  
- **Total_Trans_Ct**: Behavioral variable – total transaction count in the last 12 months  
- **Total_Ct_Chng_Q4_Q1**: Behavioral variable – change in transaction count (Q4 over Q1)  
- **Avg_Utilization_Ratio**: Utilization variable – average card utilization ratio (revolving balance ÷ credit limit)


##2. Exploratory Data Analysis

### Customer Age

In [6]:
import plotly.express as ex
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import plotly.io as pio
pio.renderers.default = 'colab'

# Plotly makes interactive graphs

# Exploring age feature distribution
fig = make_subplots(rows=2, cols=1)

tr1=go.Box(x = df['Customer_Age'], name = 'Age Box Plot', boxmean = True)
tr2=go.Histogram(x = df['Customer_Age'], name='Age Histogram')

fig.add_trace(tr1,row=1,col=1)
fig.add_trace(tr2,row=2,col=1)

fig.update_layout(height=700, width=1200, title_text="Distribution of Customer Ages")
fig.show()

Inference: The customer age distribution seems to be fairly normally distributed. Thus, we can apply the normality assumption to further utilize the age feature.

### Gender And Different Card Statuses

In [7]:
# Exploring Distribution Of Gender And Different Card Statuses
fig = make_subplots(
    rows = 2, cols = 2, subplot_titles = ('','<b>Platinum Card Holders','<b>Blue Card Holders<b>','Residuals'),
    vertical_spacing=0.09,
    specs=[[{"type": "pie","rowspan": 2}       ,{"type": "pie"}] ,
           [None                               ,{"type": "pie"}]            ,
          ]
)

fig.add_trace(
    go.Pie(values = df.Gender.value_counts().values, labels = ['<b>Female<b>','<b>Male<b>'], hole = 0.3, pull = [0,0.3]),
    row = 1, col = 1
)

fig.add_trace(
    go.Pie(
        labels = ['Female Platinum Card Holders','Male Platinum Card Holders'],
        values = df.query('Card_Category == "Platinum"').Gender.value_counts().values,
        pull = [0,0.05,0.5],
        hole = 0.3

    ),
    row = 1, col = 2
)

fig.add_trace(
    go.Pie(
        labels = ['Female Blue Card Holders','Male Blue Card Holders'],
        values = df.query('Card_Category == "Blue"').Gender.value_counts().values,
        pull = [0,0.2,0.5],
        hole = 0.3
    ),
    row = 2, col = 2
)

fig.update_layout(
    height=800,
    showlegend=True,
    title_text="<b>Distribution Of Gender And Different Card Statuses<b>",
)

fig.show()

Inference: The dataset has more 5.8% more women than men, but over all, the percentage is not higher and hence we can infer that the genders are pretty uniformly distributed.

### Dependent Count

In [8]:
# Dependent Count
fig = make_subplots(rows=2, cols=1)

tr1 = go.Box(x = df['Dependent_count'], name = 'Dependent count Box Plot',boxmean=True)
tr2 = go.Histogram(x = df['Dependent_count'], name = 'Dependent count Histogram')

fig.add_trace(tr1,row = 1, col = 1)
fig.add_trace(tr2, row = 2, col = 1)

fig.update_layout(height = 700, width = 1200, title_text = "Distribution of Dependent counts (close family size)")
fig.show()

Inference: Even this distribution of Dependent counts is fairly normally distributed with a slight right skew.

### Education Level

In [10]:
ex.pie(df, names = 'Education_Level', title = 'Propotion Of Education Levels', hole = 0.33)

Inference: On the assumption that unknown status lack education, we can still say that more than 70% of the customers have a formal education level. About 35% of customer have a higher level of education.

## Marital Status

In [11]:
ex.pie(df, names = 'Marital_Status', title = 'Propotion Of Marital Status', hole = 0.33)

Inference: Almost half of the bank customers are married and interestingly enough, almost the entire half are single. Only about 7% of the customers are divorced, which is surprising considering the worldwide divorce rate statistics!

### Income Status

In [12]:
ex.pie(df, names = 'Income_Category', title = 'Propotion Of Income Categories', hole = 0.33)

Illustration of various Income categories of the bank customers. It will be interesting to know the correlation of income and their churn, defaults status. There's a high percentage of the customers earning less than 40k!

In [14]:
# Analyze the distribution of Income_Category
income_counts = df['Income_Category'].value_counts()
print("Income Category Distribution:")
print(income_counts)

# Visualize the distribution using Plotly
fig = ex.bar(x=income_counts.index, y=income_counts.values,
              labels={'x': 'Income Category', 'y': 'Frequency'},
              title='Distribution of Income Categories')
fig.update_layout(xaxis={'categoryorder':'total descending'}) # Optional: order bars by frequency
fig.show()

# A simple visual inspection for Power Law characteristics:
# Check if there's a clear "long tail" - a few categories with high counts
# and many with low counts.
# Note: A formal test for Power Law distribution would require more sophisticated methods
# like fitting the data to a Power Law function and evaluating the fit.

Income Category Distribution:
Income_Category
Less than $40K    3561
$40K - $60K       1790
$80K - $120K      1535
$60K - $80K       1402
Unknown           1112
$120K +            727
Name: count, dtype: int64


I was curious to see if it follows a Power Law distribution or Log Normal distribution, but I skipped that such distributions need to have contnuous values rather than values in range.

### Card Category

In [15]:
ex.pie(df, names = 'Card_Category', title = 'Propotion Of Card Categories', hole = 0.33)

In [19]:
# Calculate the frequency of each card category
card_counts = df['Card_Category'].value_counts()
print("Card Category Distribution:")
print(card_counts)

# Visualize the distribution using Plotly (Bar Plot)
fig = ex.bar(x=card_counts.index, y=card_counts.values,
              labels={'x': 'Card Category', 'y': 'Frequency'},
              title='Distribution of Card Categories')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

Card Category Distribution:
Card_Category
Blue        9436
Silver       555
Gold         116
Platinum      20
Name: count, dtype: int64


In [20]:
# Calculate the total number of customers
total_customers = card_counts.sum()

# Calculate the percentage of customers in each card category
card_percentages = (card_counts / total_customers) * 100

print("Card Category Distribution (Percentages):")
print(card_percentages)

# Calculate the percentage of 'Blue' cardholders
blue_percentage = card_percentages['Blue']

# Calculate the percentage of all other cardholders
other_percentages = card_percentages.drop('Blue').sum()

print(f"\nPercentage of 'Blue' cardholders: {blue_percentage:.2f}%")
print(f"Percentage of other cardholders: {other_percentages:.2f}%")

if blue_percentage >= 80 and other_percentages <= 20:
    print("\nBased on these percentages, the distribution for 'Blue' vs. other categories is close to the 80-20 rule.")
else:
    print("\nBased on these percentages, the distribution for 'Blue' vs. other categories does not strictly follow the 80-20 rule.")

Card Category Distribution (Percentages):
Card_Category
Blue        93.176656
Silver       5.480399
Gold         1.145453
Platinum     0.197492
Name: count, dtype: float64

Percentage of 'Blue' cardholders: 93.18%
Percentage of other cardholders: 6.82%

Based on these percentages, the distribution for 'Blue' vs. other categories is close to the 80-20 rule.


### Months how long the customers have been a part of the bank

In [24]:
# Distribution of months of how long the customer has been a part of the bank
fig = make_subplots(rows = 2, cols = 1)

tr1 = go.Box(x = df['Months_on_book'], name = 'Months on book Box Plot', boxmean = True)
tr2 = go.Histogram(x = df['Months_on_book'], name = 'Months on book Histogram')

fig.add_trace(tr1, row = 1, col = 1)
fig.add_trace(tr2, row = 2, col = 1)

fig.update_layout(height = 700, width = 1200, title_text = "Distribution based on how long the customer has been a part of the bank")
fig.show()

Even though the distribution appeared to be more spread out, we cannot make assumption for normality by visualization alone. Hence, we will now examine kurtosis and skewness to determine the normality of this variable.

In [26]:
# The textbook definition usually has the values compared to =, > or < 3,
# But, kurt() gives excess Kurtosis (using Fischer's defn of Kurtosis)
# So, values closer to 0 are normally distributed.
print('Kurtosis of Months on book features is : {}'.format(df['Months_on_book'].kurt()))

Kurtosis of Months on book features is : 0.40010012019986707


For reference:

**Kurtosis** tells us about the "tailedness" and peakedness of a distribution compared to a normal distribution.

*   **Mesokurtic**: Similar to normal (kurtosis close to 0, or 3 for standard kurtosis).
*   **Leptokurtic**: Higher kurtosis (>0), fatter tails, sharper peak.
*   **Platykurtic**: Lower kurtosis (<0), thinner tails, flatter peak.

**Skewness** measures how symmetrical a distribution is. It tells us if the data is leaning to one side.

*   **Zero Skewness:** Symmetric distribution (like a bell curve). Skewness value is close to 0.
*   **Positive Skewness:** The tail is longer on the right side. Data is clustered on the left. Skewness value is positive (> 0).
*   **Negative Skewness:** The tail is longer on the left side. Data is clustered on the right. Skewness value is negative (< 0).

In [27]:
# Calculate the skewness of the 'Months_on_book' feature
skewness = df['Months_on_book'].skew()

print(f"The skewness of 'Months_on_book' is: {skewness:.4f}")

The skewness of 'Months_on_book' is: -0.1066


Inference: Based on a kurtosis of 0.4 (mesokurtic) and skewness of -0.1066 (nearly symmetric), along with visual inspection, the 'Months_on_book' distribution is not perfectly normal but is reasonably close. It can likely be treated as approximately normally distributed for most analyses. (This serves as a perfect reminder to not rely on visualizations alone!)